In [1]:
import numpy as np
import pandas as pd
import glob
import argparse
import os
from pathlib import Path
from misc.joinfile import joinfile

pd.set_option('display.max_colwidth', None)

mapping_path = '/home/brainstimmaps/RESEARCH/20xx_Projects/2009_DeepMaps01/deepmaps/data/raw/tables/bern+cologne/mapping/multicentricTableBernCologneImprovedOnlyRev05Patient21Excluded.csv'
output_folder = '/home/brainstimmaps/RESEARCH/20xx_Projects/2009_DeepMaps01/deepmaps/data/raw/tables'
vtas_path = '/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped'

In [2]:
def set_interp(df : pd.DataFrame):
    # linear interpolation 
    #print()
    #print(name, end=' : ')
    groups = df.groupby(['leadID', 'contactID'])
    for name, group in groups:
        #print(name)
        #print(group[['amplitude', 'mapping']])
        # if there are mapping values in the group
        # [2,4] -> [0.17, 0.33]
        # excepted : [1, 2, 3, 4, 5] -> [0.085, 0.17, 0.25, 0.33, 0.41]
        if not group[group['mapping'] == 1].empty:
            first_amp = group[group['mapping'] == 1]['amplitude'].to_list()[0]
            score = group[group['mapping'] == 1]['relativeImprovement'].to_list()[0]
            #print(max_amp, end = '')
                        
            # max amplitude at 3 -> score = 0.6
            xp = (0., first_amp)
            fp = (0., score)
            amplitudes_interp = group[group['amplitude'] <= first_amp]['amplitude'].to_list()
            amplitudes_left = group[group['amplitude'] > first_amp]['amplitude'].to_list()
            #print(amplitudes)
            interp = np.interp(amplitudes_interp, xp, fp)
            interp = np.append(interp, amplitudes_left)
            #print(interp)
            #print(df.loc[((df['leadID'] == name[0]) & ([df['contactID'] == name[1]])),'relativeImprovement'])
            #break
            a = df['leadID'] == name[0]
            b = df['contactID'] == name[1]
            df.loc[a & b,'relativeImprovement'] = interp
            #print(df.loc[a & b,'relativeImprovement'])
            #group['relativeImprovement'] = interp
            
            #print(df.query(f'leadID == {name[0]} and contactID == {name[1]}')['relativeImprovement'])
            
            #print(group[['amplitude', 'relativeImprovement', 'mapping']])
    return df


def set_step(df : pd.DataFrame):
    #print()
    #print(name, end=' : ')
    #print(group[['amplitude', 'mapping']])
    groups = df.groupby(['leadID', 'contactID'])
    for name, group in groups:
        if not group[group['mapping'] == 1].empty:
            a = df['leadID'] == name[0]
            b = df['contactID'] == name[1]
            df.loc[a & b,'relativeImprovement'] = group['relativeImprovement'].interpolate(method='pad')

        #print(group[['amplitude', 'relativeImprovement', 'mapping']])
    df['relativeImprovement'] = df['relativeImprovement'].fillna(0.0)
    return df

def get_idx_to_del(df : pd.DataFrame):
    idx_to_del_part, idx_to_del_full = [], []
    groups = df.groupby(['leadID', 'contactID'])
    for name, group in groups:
        #print()
        #print(name, end=' : ')
        #print(group[['amplitude', 'mapping']])
        if group[group['mapping'] == 1].empty:
            idx_buff = group.index
            idx_to_del_full.extend(idx_buff)
            idx_to_del_part.extend(idx_buff)
            #print('no mapping here', end = '')
            #print(group_.index)
        else:
            max_amp = group[group['mapping'] == 1]['amplitude'].to_list()

            idx_buff_full = group[(group['amplitude'].isin(max_amp)) & (group['mapping'] == 0)].index
            idx_to_del_full.extend(idx_buff_full)
            
            idx_buff_part = group[(group['amplitude'] >= max(max_amp)) & (group['mapping'] == 0)].index
            idx_to_del_part.extend(idx_buff_part)
            #print(idx_buff_part)
    return idx_to_del_part, idx_to_del_full


In [7]:
# --- GETTING ALL THE FILEPATHS IN THIS FORMAT AND MERGING LEFT & RIGHT ---
flipped_list = glob.glob('/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/*/stimulations/MNI_ICBM_2009b_NLIN_ASYM/*/*.nii')  
#print(flipped_list)
# --- DICT TO CONVERT CONTACT LEAD NAMING CONVERTIONS ---------------------
medtronic_to_leaddbs = {
    '0' : '0',
    '1' : '16',
    '2' : '17',
    '3' : '7',
    '8' : '8',
    '9' : '18',
    '10' : '19',
    '11' : '15'
}

flipped_list = [x for x in flipped_list if '21' not in x.split('/')[-5]] #excluding patient 21
flipped_list.sort()

# --- MAKING ROW ENTRIES ACCORDING TO leaddbs SAVARY'S PSM CLASS ----------
centerID, leadID, contactID, verciseID, relativeImprovement, amplitude, efficiency, leadModel = [], [], [], [], [], [], [], []

for item in flipped_list:
    print(item)
    # '/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/02d_CologneFlippedNotResliced/*/stimulations/MNI_ICBM_2009b_NLIN_ASYM/*/*.nii.gz'
    patientID = item.split('/')[-5] #TWEED## or #

    hemisphere = 'right' if item.split('/')[-1].split('.')[0] == 'vat_right' else 'left'

    if 'TWEED' in patientID: # Boston scientific lead model
        patientID = 230 + int(patientID[-2:])
        boston = True
        leadModel.append('Boston Scientific Vercise')
        contact = item.split('/')[-2].split('_')[0][1:]
        if hemisphere == 'right':
            contact = contact.split('-')[0]
        else:
            contact = contact.split('-')[1]
    else: # Medtronic lead model
        patientID = 200 + int(patientID)
        boston = False
        
        if patientID == 221:
            continue
        contact = item.split('/')[-2].split('_')[0]
        leadModel.append('Medtronic 3389')
    
    centerID.append(2)   

    leadID.append(patientID + (0.5 if hemisphere == 'left' else 0.0)) #patientID (without zeros) + 0.5 if left
    
    if not boston:
        vercise = medtronic_to_leaddbs[contact]
    else:
        vercise = contact

    #contact = contact_flipped_to_left[contact] if hemisphere == 'right' else contact
    contactID.append(int(contact)) #smaller = left, bigger = right
    verciseID.append(int(vercise))

    relativeImprovement.append(np.nan) # 0 by default, TODO : interpolation from full-effect (verify with Koln dataset)

    amplitude_ = item.split('/')[-2].split('_')[1]
    amplitude.append(float(amplitude_)) #in name
    
    efficiency.append(np.nan) # 0 by default, TODO : infer from interpolation 

#print(len(centerID), len(leadModel), len(contactID), len(relativeImprovement), len(amplitude), len(efficiency), len(flipped_list))
dict_infer = {
    'centerID' : centerID,
    'leadModel' : leadModel,
    'leadID' : leadID,
    'contactID' : contactID,
    'verciseID' : verciseID,
    'relativeImprovement' : relativeImprovement,
    'amplitude' : amplitude,
    'efficiency' : efficiency,
    'filename' : flipped_list}
df_infer = pd.DataFrame(dict_infer, index = list(range(len(flipped_list))))


/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_0.5_final_033014_02/vat_right.nii
/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_1.5_final_033014_02/vat_right.nii
/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_1_final_033014_02/vat_right.nii
/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_2.5_final_033014_02/vat_right.nii
/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_2_final_033014_02/vat_right.nii
/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_3.5_final_033014_02/vat_right.nii
/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM

In [8]:
df_infer.head(20)

,centerID,leadModel,leadID,contactID,verciseID,relativeImprovement,amplitude,efficiency,filename
0,2,Medtronic 3389,201.0,0,0,NaN,0.5,NaN,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_0.5_final_033014_02/vat_right.nii
1,2,Medtronic 3389,201.0,0,0,NaN,1.5,NaN,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_1.5_final_033014_02/vat_right.nii
2,2,Medtronic 3389,201.0,0,0,NaN,1.0,NaN,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_1_final_033014_02/vat_right.nii
3,2,Medtronic 3389,201.0,0,0,NaN,2.5,NaN,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_2.5_final_033014_02/vat_right.nii
4,2,Medtronic 3389,201.0,0,0,NaN,2.0,NaN,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_2_final_033014_02/vat_right.nii
5,2,Medtronic 3389,201.0,0,0,NaN,3.5,NaN,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_3.5_final_033014_02/vat_right.nii
6,2,Medtronic 3389,201.0,0,0,NaN,3.0,NaN,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_3_final_033014_02/vat_right.nii
7,2,Medtronic 3389,201.0,0,0,NaN,4.0,NaN,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_4_final_033014_02/vat_right.nii
8,2,Medtronic 3389,201.0,0,0,NaN,5.0,NaN,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/0_5_final_033014_02/vat_right.nii
9,2,Medtronic 3389,201.5,10,19,NaN,1.0,NaN,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/10_1_final_033014_02/vat_flipped.nii


In [9]:


# # --- EXPORTING FULL INFERRED TABLE ---------------------------------------
# output_path = joinfile(output_folder, 'inferred/nan_interp/table_psm.csv')
# Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
# df_infer.to_csv(output_path)

df_infer['mapping'] = 0
output_path = joinfile(output_folder, 'inferred/nan_interp/table_mapping.csv')
Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
df_infer.to_csv(output_path)

# --- IMPORTING MAPPING TABLE ---------------------------------------------
df_map = pd.read_csv(mapping_path)
df_map = df_map[df_map['centerID'] == 2] # selecting only koln
df_map['leadModel'] = df_map['leadID']
df_map['verciseID'] = df_map['contactID']
df_map.loc[df_map['leadID'] <= 225.5, 'leadModel'] = 'Medtronic 3389'
df_map.loc[df_map['leadID'] > 225.5, 'leadModel'] = 'Boston Scientific Vercise'
df_map.loc[df_map['leadModel'] == 'Medtronic 3389', 'verciseID'] = df_map.loc[df_map['leadModel'] == 'Medtronic 3389', 'verciseID'].astype(str).map(medtronic_to_leaddbs)


In [10]:
# --- CHANGING MAPPING FILENAME TO ABSOLUTE PATHS -------------------------
mapping_filenames = df_map['filename']
abs_filenames = []
for filename in mapping_filenames:
# old : ../../03_Data/VTAsForMulticentricPSMRev05/CologneVTAsResliced/208_2_3.nii
# new : media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/02d_CologneFlippedNotResliced/8/stimulations/MNI_ICBM_2009b_NLIN_ASYM/2_3_final_033014_02/vat_right.nii.gz
    file_short = filename.split('/')[-1]
    patientID_map = str(int(file_short.split('_')[0]) - 200)
    contactID_map = file_short.split('_')[1]
    amplitude_map = file_short.split('_')[2][0]
    side_map = 'right' if int(contactID_map) in (0,1,2,3,4,5,6,7,16,17) else 'flipped'
    end_path = patientID_map + '/stimulations/MNI_ICBM_2009b_NLIN_ASYM/' + \
            contactID_map + '_' + amplitude_map + '_final_033014_02/vat_' + side_map + '.nii'

    abs_filenames.append(joinfile(vtas_path, end_path))

df_map['filename'] = abs_filenames

In [11]:
df_map.head()

,centerID,leadID,contactID,relativeImprovement,amplitude,efficiency,filename,leadModel,verciseID
660,2,201.5,8,0.50,2.0,0.250,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/8_2_final_033014_02/vat_flipped.nii,Medtronic 3389,8
661,2,201.5,8,0.75,3.0,0.250,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/8_3_final_033014_02/vat_flipped.nii,Medtronic 3389,8
662,2,201.5,9,0.50,2.0,0.250,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/9_2_final_033014_02/vat_flipped.nii,Medtronic 3389,18
663,2,201.5,9,0.75,3.0,0.250,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/9_3_final_033014_02/vat_flipped.nii,Medtronic 3389,18
664,2,201.5,11,0.50,4.0,0.125,/media/brainstimmaps/DATA/2009_DeepMaps01/03_Data/03_MassiveCologneFlipped/1/stimulations/MNI_ICBM_2009b_NLIN_ASYM/11_4_final_033014_02/vat_flipped.nii,Medtronic 3389,15


In [12]:
df_map.to_csv('mapping_cologne.csv', index=False)

In [13]:

# --- ERASING ENTRIES ON INFERRED TABLE -----------------------------------
df_map['mapping'] = 1
df_merged = pd.concat([df_infer,df_map], ignore_index=True)

df_merged = df_merged.astype({'amplitude': 'float64'})
df_merged = df_merged.sort_values(by=['leadID', 'contactID', 'amplitude'])

df_merged.loc[df_merged['mapping'] == 0, 'relativeImprovement'] = pd.NA
df_merged['leadID'] = df_merged['leadID'].astype(str)
df_merged['contactID'] = df_merged['contactID'].astype(str)


In [14]:
df_merged.to_csv('merged_cologne.csv', index=False)

In [15]:
def extended_interp(x, xp, fp):
    result = []
    for val in x:
        if val <= xp[-1]:
            result.append(np.interp(val, xp, fp))
        else:
            slope = (fp[-1] - fp[-2]) / (xp[-1] - xp[-2])
            result.append(fp[-1] + slope * (val - xp[-1]))
    return np.asarray(result)


def lin_interp_group(group):
    #print(group)
    # Find the indices where 'mapping' column is 1
    # Step 6: Sort the DataFrame by column E in descending order
    group = group.sort_values(by='mapping', ascending=False)
    # Step 7: Remove duplicate rows based on columns C, D, and E within each group
    group = group.drop_duplicates(subset=['leadID', 'contactID', 'amplitude'], keep='first')

    group = group.sort_values(by='amplitude', ascending=True)

    mask = group['mapping'] == 1
    indices = group.index[mask]

    if len(indices) == 0:
        # Return an empty DataFrame if no ground truth values found
        return pd.DataFrame()

    # Interpolate values based on ground truth points
    # Create an array of consecutive numbers as x-values
    x_values = group.loc[indices, 'amplitude'].to_list()
    y_values = group.loc[indices, 'relativeImprovement'].to_list()
    x_full = group.loc[:, 'amplitude'].to_list()
    # Map the sorted indices to the corresponding x-values
    x_values.insert(0, 0)
    y_values.insert(0, 0)
    # Interpolate values based on ground truth points

    interpolated_values = extended_interp(x_full, x_values, y_values)

    # Replace interpolated values that exceed 1 with 1 and below 0 with 0
    interpolated_values = np.where(interpolated_values > 1, 1, interpolated_values)
    interpolated_values = np.where(interpolated_values < 0, 0, interpolated_values)

    # Assign interpolated values to the 'relativeImprovement' column
    group['relativeImprovement'] = interpolated_values

    return group

def step_interp_group(group):

    group = group.sort_values(by='mapping', ascending=False)
    group = group.drop_duplicates(subset=['leadID', 'contactID', 'amplitude'], keep='first')
    group = group.sort_values(by='amplitude', ascending=True)

    mask = group['mapping'] == 1
    indices = group.index[mask]

    if len(indices) == 0:
        # Return an empty DataFrame if no ground truth values found
        return pd.DataFrame()
    
    group['relativeImprovement'] = group['relativeImprovement'].interpolate(method='pad')

    group['relativeImprovement'] = group['relativeImprovement'].fillna(0.0)
    print(group['relativeImprovement'])
    return group


Missing : origin -> first GT point, last GT point -> same inter-slope

In [16]:

df_merged_interp = df_merged.copy(deep=True)
df_merged_step = df_merged.copy(deep=True)
#df_merged_step.loc[(df_merged_step.amplitude == 1.0) & (df_merged_step.mapping == 0),'relativeImprovement'] = 0.0


idx_to_del_part, idx_to_del_full = get_idx_to_del(df_merged_interp)
df_aug_step_part = df_merged_step.drop(idx_to_del_part)
df_aug_interp_part = df_merged_interp.drop(idx_to_del_part)
df_aug_step_full = df_merged_step.drop(idx_to_del_full)
df_aug_interp_full = df_merged_interp.drop(idx_to_del_full)


grouped_interp_part = df_aug_interp_part.groupby(['leadID', 'contactID'])
df_aug_interp_part = grouped_interp_part.apply(lin_interp_group)
df_aug_interp_part = df_aug_interp_part.reset_index(drop=True)

grouped_interp_full = df_aug_interp_full.groupby(['leadID', 'contactID'])
df_aug_interp_full = grouped_interp_full.apply(lin_interp_group)
df_aug_interp_full = df_aug_interp_full.reset_index(drop=True)


grouped_step_part = df_aug_step_part.groupby(['leadID', 'contactID'])
df_aug_step_part = grouped_step_part.apply(step_interp_group)
df_aug_step_part = df_aug_step_part.reset_index(drop=True)

grouped_step_full = df_aug_step_full.groupby(['leadID', 'contactID'])
df_aug_step_full = grouped_step_full.apply(step_interp_group)
df_aug_step_full = df_aug_step_full.reset_index(drop=True)

#print(df_merged_step[['amplitude', 'relativeImprovement', 'mapping']])
        
df_aug_interp_full['efficiency'] = df_aug_interp_full['relativeImprovement'] / df_aug_interp_full['amplitude']
df_aug_interp_part['efficiency'] = df_aug_interp_part['relativeImprovement'] / df_aug_interp_part['amplitude']
df_aug_step_full['efficiency'] = df_aug_step_full['relativeImprovement'] / df_aug_step_full['amplitude']
df_aug_step_part['efficiency'] = df_aug_step_part['relativeImprovement'] / df_aug_step_part['amplitude']

#print(df_augmented_interp_full[['amplitude', 'relativeImprovement', 'mapping']])

#df_merged_interp.to_csv('new_lin_interp_test.csv',index=False)
#df_merged_step.to_csv('new_step_interp_test.csv',index=False)

14      0.0
15      0.0
16      0.0
1804    0.5
Name: relativeImprovement, dtype: float64
34      0.00
1800    0.50
1801    0.75
Name: relativeImprovement, dtype: float64
39      0.00
1802    0.50
1803    0.75
Name: relativeImprovement, dtype: float64
1805    0.333333
Name: relativeImprovement, dtype: float64
560     0.000000
1806    0.166667
562     0.166667
1807    0.333333
Name: relativeImprovement, dtype: float64
575     0.0
576     0.0
577     0.0
1808    0.2
Name: relativeImprovement, dtype: float64
580     0.000000
1809    0.285714
1810    0.142857
1811    0.142857
1812    0.428571
Name: relativeImprovement, dtype: float64
1813    0.25
1814    0.25
Name: relativeImprovement, dtype: float64
1815    0.2
Name: relativeImprovement, dtype: float64
640     0.000000
641     0.000000
642     0.000000
1816    0.166667
Name: relativeImprovement, dtype: float64
1817    0.166667
656     0.166667
1818    0.333333
1819    0.500000
1820    0.500000
Name: relativeImprovement, dtype: float64
660

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, widgets

n_groups = len(df_aug_interp_full.groupby(['leadID', 'contactID']).groups.keys()) - 1

@widgets.interact(group_index=widgets.IntSlider(min=0, max=n_groups, step=1, value=0),
                  df_index=widgets.IntSlider(min=0, max=3, step=1, value=0))
def plot_group(group_index, df_index):
    # Getting the corresponding group
    dfs = [df_aug_step_part, df_aug_step_full, df_aug_interp_part, df_aug_interp_full]
    df_str = ['step part', 'step full', 'lin part', 'lin full']
    # Grouping the dataframe
    grouped_df = dfs[df_index].groupby(['leadID', 'contactID'])

    # Extracting the unique group indices
    group_indices = grouped_df.groups.keys()
    group_indices = sorted(group_indices)

    group = group_indices[group_index]
    data = grouped_df.get_group(group)

    # Plotting the data with a black line connecting the points and the origin
    plt.figure(figsize=(10, 6))
    # Plotting points where mapping = 1 in red, others in blue
    plt.scatter(data[data['mapping'] == 1]['amplitude'], data[data['mapping'] == 1]['relativeImprovement'],
                color='red', marker='o', label='Mapping')
    plt.scatter(data[data['mapping'] != 1]['amplitude'], data[data['mapping'] != 1]['relativeImprovement'],
                color='blue', marker='o', label='Inferred')

    # Add origin point
    origin = (0, 0)
    plt.plot(origin[0], origin[1], marker='o', markersize=8, color='green', label='Origin')

    # Connect the origin to the first point
    first_point = data.iloc[0]
    plt.plot([origin[0], first_point['amplitude']], [origin[1], first_point['relativeImprovement']],
             color='black', linestyle='--', alpha=0.5)

    # Connect the points
    for i in range(len(data) - 1):
        plt.plot([data.iloc[i]['amplitude'], data.iloc[i+1]['amplitude']],
                 [data.iloc[i]['relativeImprovement'], data.iloc[i+1]['relativeImprovement']],
                 color='black', linestyle='--', alpha=0.5)

    plt.xlabel('Amplitude')
    plt.ylabel('Relative Improvement')
    plt.title(f'Amplitude vs Relative Improvement (Group: {group}, Augmentation : {df_str[df_index]})')
    plt.grid(True)
    plt.legend()
    plt.show()


interactive(children=(IntSlider(value=0, description='group_index', max=119), IntSlider(value=0, description='…

Step interpolation behaves strangely for the 1st cohort. Step part didnt erase the non-mapping values.

In [ ]:

# --- EXPORT IN PSM FORMAT AND WITH MAPPING INFORMATION -------------------

# augmented step part
output_path = joinfile(output_folder, 'cologne/augmented_part/step_interp/table_mapping.csv')
Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
df_aug_step_part.to_csv(output_path)

# df_augmented_step_part_psm = df_augmented_step_part.drop('mapping', axis=1)
# output_path = joinfile(output_folder, 'cologne/augmented_part/step_interp/table_psm.csv')
# Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
# df_augmented_step_part_psm.to_csv(output_path)

# augmented interp part
output_path = joinfile(output_folder, 'cologne/augmented_part/linear_interp/table_mapping.csv')
Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
df_aug_interp_part.to_csv(output_path)

# df_augmented_interp_part_psm = df_augmented_interp_part.drop('mapping', axis=1)
# output_path = joinfile(output_folder, 'cologne/augmented_part/linear_interp/table_psm.csv')
# Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
# df_augmented_interp_part_psm.to_csv(output_path)

# augmented step full
output_path = joinfile(output_folder, 'cologne/augmented_full/step_interp/table_mapping.csv')
Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
df_aug_step_full.to_csv(output_path)

# df_augmented_step_full_psm = df_augmented_step_full.drop('mapping', axis=1)
# output_path = joinfile(output_folder, 'cologne/augmented_full/step_interp/table_psm.csv')
# Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
# df_augmented_step_full_psm.to_csv(output_path)

# augmented interp full
output_path = joinfile(output_folder, 'cologne/augmented_full/linear_interp/table_mapping.csv')
Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
df_aug_interp_full.to_csv(output_path)

# df_augmented_interp_full_psm = df_augmented_interp_full.drop('mapping', axis=1)
# output_path = joinfile(output_folder, 'cologne/augmented_full/linear_interp/table_psm.csv')
# Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
# df_augmented_interp_full_psm.to_csv(output_path)

# mapping
output_path = joinfile(output_folder, 'cologne/mapping/no_interp/table_mapping.csv')
Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
df_map.to_csv(output_path)

# df_map_psm = df_map.drop('mapping', axis=1)
# output_path = joinfile(output_folder, 'cologne/mapping/no_interp/table_psm.csv')
# Path(output_path).parents[0].mkdir(parents=True, exist_ok=True)
# df_map_psm.to_csv(output_path)

